In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
# !pip install torch
# !pip install torchmetrics

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-g61ijyv0
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-g61ijyv0
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 881.9/881.9 MB 90.5 MB/s eta 0:0

In [ ]:
# gpu 프리미엄 사용 시
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 197.1 MB/s eta 0:00:01tcmalloc: large alloc 2041339904 bytes == 0x2c36000 @  0x7f7356b771e7 0x4d30a0 0x4d312c 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 191.6 MB/s eta 0:00:01tcmalloc: large alloc 2551676928 bytes == 0x7c6fc000 @  0x7f7356b78615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd
from tqdm.notebook import tqdm

import re

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

from transformers import AutoTokenizer, AutoModelForSequenceClassification


using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


### 데이터 전처리


In [ ]:
import pandas as pd

# 데이터 입력


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/ekonlpy_kia_test_1225_sentsplit.csv')
df['kmm_label_n_n5'] = df['kmm_label_n_n5'] + 2
df

,media,date,url,content,target,mpc_pos,mpc_neg,mpck_token,score_diff,score_diff+tar,pos_neg,pos_neg_tardiff,kmm_label,kmm_label_n5,kmm_label_n_n5
0,연합인포맥스,20171201,http://news.einfomax.co.kr/news/articleView.ht...,국내완성차 11월 판매 76만2천967대전년비 8.4서울연합인포맥스 이민재 기자 ...,-1,9.986208e-01,0.001379,판매 [SEP] 기자 업체 대비 다소 부진 판매 실적 기록 하 [SEP] 업계 따르...,0.997242,9.972415e-01,pos,pos,2,상승,3
1,연합뉴스,20171204,http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.as...,"검찰, 에 건넨 특활비 추적해 유의미한 용처 파악서울연합뉴스 방현덕 기자 박근혜 ...",0,1.078517e-01,0.892148,건넨 특활 비 추적 유의미한 용처 파악 방 현덕 기자 시절 특수 활동비 상납 하 혐...,-0.784297,-2.842967e-01,neg,neg,1,하락,1
2,이데일리,20171204,http://www.edaily.co.kr/news/newspath.asp?news...,특징주현대차 3인방 내림세.. 판매부진에 파업 우려이데일리 함정선 기자 현대차그룹 ...,0,8.480174e-07,0.999999,특징 주현 대차 인방 내림 [SEP] 판매 부진 파업 우려 함정 선 기자 인방 주가...,-0.999998,-4.999983e-01,neg,neg,1,하락,1
3,이데일리,20171204,http://www.edaily.co.kr/news/newspath.asp?news...,제네시스 80 디젤 출격임박수입차 수준 연비성능 뽑았다이데일리 노재웅 기자 의 고급...,0,2.486862e-07,1.000000,출격 임박 수준 연비 성능 뽑 재웅 기자 브랜드 탑재 하 출시 하 예정 [SEP] ...,-1.000000,-4.999995e-01,neg,neg,1,하락,1
4,이데일리,20171204,http://www.edaily.co.kr/news/newspath.asp?news...,이데일리 11월 건설 체감경기 악화정책금리 인상 영향이데일리 1. 최종구 창업중소기...,0,5.016498e-02,0.949835,건설 체감 경기 악화 금리 인상 영향 [SEP] 창업 중소기업 애로 적극 해소 노력...,-0.899670,-3.996700e-01,neg,neg,1,하락,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11521,이데일리,20221130,http://www.edaily.co.kr/news/newspath.asp?news...,"처벌로 못 줄인 산재정부 산재 정책, 기업 자율규제 중심으로 바뀐다이데일리 최정훈 ...",1,1.045173e-16,1.000000,처벌 못 줄 산재 산재 정책 기업 자율규제 중심 바뀐다 기자 달하 산재 사고 사망자...,-1.000000,1.110223e-16,neg,pos,3,중간,2
11522,이데일리,20221130,http://www.edaily.co.kr/news/newspath.asp?news...,프로필루크 동커볼케 현대자동차 사장이데일리 손의연 기자 그룹이 30일 최고창조책임자...,1,3.755127e-01,0.624487,프로필 동 크 손 연 기자 그룹 최고 창조 책임자 동 크 부사장 승진 시키 장단 단...,-0.248975,7.510254e-01,neg,pos,2,상승,3
11523,이데일리,20221130,http://www.edaily.co.kr/news/newspath.asp?news...,"안정과 미래 동시에 챙긴 정의선, 현대차그룹 핀셋 인사종합이데일리 송승현 기자 올해...",1,6.952240e-08,1.000000,안정 미래 챙긴 핀셋 종합 기자 취임 차 맞 경영 안정화 미래 사업경쟁력 강화 꾀하...,-1.000000,1.390448e-07,neg,pos,3,중간,2
11524,이데일리,20221129,http://www.edaily.co.kr/news/newspath.asp?news...,"코스피, 경기둔화 우려에 하락 출발2400선 턱걸이이데일리 양지윤 기자 코스피지수...",1,2.288448e-01,0.771155,코스피 경기 둔화 우려 하락 출발 선 턱걸이 기자 코스피 하락 출발 하 [SEP] ...,-0.542310,4.576896e-01,neg,pos,3,중간,2


In [ ]:
# 날짜 긍부정 targetpm 라벨링
df_pm_dict = {}
df_pm = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/kia_new_label_sentsplit_news_label_1231.csv')
df_date = df_pm.groupby('date')[['targetpm']].mean()
for idx in df_date.index :
  str_idx = ''.join(idx.split('-'))
  df_pm_dict[str_idx] = int(df_date.loc[idx, 'targetpm'])
df_pm_dict

{'20171204': 1,
 '20171205': 3,
 '20171206': 3,
 '20171207': 1,
 '20171208': 1,
 '20171211': 1,
 '20171212': 3,
 '20171213': 3,
 '20171214': 1,
 '20171215': 3,
 '20171218': 3,
 '20171219': 1,
 '20171220': 3,
 '20171221': 1,
 '20171222': 3,
 '20171226': 3,
 '20171227': 3,
 '20171228': 3,
 '20180102': 1,
 '20180103': 1,
 '20180104': 1,
 '20180105': 3,
 '20180108': 3,
 '20180109': 3,
 '20180110': 3,
 '20180111': 1,
 '20180112': 1,
 '20180115': 1,
 '20180116': 3,
 '20180117': 3,
 '20180118': 1,
 '20180119': 3,
 '20180122': 1,
 '20180123': 3,
 '20180124': 3,
 '20180125': 3,
 '20180126': 1,
 '20180129': 3,
 '20180130': 3,
 '20180131': 3,
 '20180201': 1,
 '20180202': 3,
 '20180205': 1,
 '20180206': 1,
 '20180207': 1,
 '20180208': 1,
 '20180209': 1,
 '20180212': 1,
 '20180213': 1,
 '20180214': 3,
 '20180219': 3,
 '20180220': 1,
 '20180221': 3,
 '20180222': 1,
 '20180223': 3,
 '20180226': 3,
 '20180227': 3,
 '20180228': 1,
 '20180302': 1,
 '20180305': 1,
 '20180306': 3,
 '20180307': 1,
 '201803

In [ ]:
df2 = df.copy()
df2['date'] = df2['date'].astype('str')
df2 = df2[df2['date'].isin(list(df_pm_dict.keys()))]
df2 = df2.reset_index(drop=True)
df2['targetpm'] = df2['date'].apply(lambda x : df_pm_dict[x])
df2['target'] = df2['target'] + 2
df2.head(20)

,media,date,url,content,target,mpc_pos,mpc_neg,mpck_token,score_diff,score_diff+tar,pos_neg,pos_neg_tardiff,kmm_label,kmm_label_n5,kmm_label_n_n5,targetpm
0,연합뉴스,20171204,http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.as...,"검찰, 에 건넨 특활비 추적해 유의미한 용처 파악서울연합뉴스 방현덕 기자 박근혜 ...",2,1.078517e-01,0.892148,건넨 특활 비 추적 유의미한 용처 파악 방 현덕 기자 시절 특수 활동비 상납 하 혐...,-0.784297,-2.842967e-01,neg,neg,1,하락,1,1
1,이데일리,20171204,http://www.edaily.co.kr/news/newspath.asp?news...,특징주현대차 3인방 내림세.. 판매부진에 파업 우려이데일리 함정선 기자 현대차그룹 ...,2,8.480174e-07,0.999999,특징 주현 대차 인방 내림 [SEP] 판매 부진 파업 우려 함정 선 기자 인방 주가...,-0.999998,-4.999983e-01,neg,neg,1,하락,1,1
2,이데일리,20171204,http://www.edaily.co.kr/news/newspath.asp?news...,제네시스 80 디젤 출격임박수입차 수준 연비성능 뽑았다이데일리 노재웅 기자 의 고급...,2,2.486862e-07,1.000000,출격 임박 수준 연비 성능 뽑 재웅 기자 브랜드 탑재 하 출시 하 예정 [SEP] ...,-1.000000,-4.999995e-01,neg,neg,1,하락,1,1
3,이데일리,20171204,http://www.edaily.co.kr/news/newspath.asp?news...,이데일리 11월 건설 체감경기 악화정책금리 인상 영향이데일리 1. 최종구 창업중소기...,2,5.016498e-02,0.949835,건설 체감 경기 악화 금리 인상 영향 [SEP] 창업 중소기업 애로 적극 해소 노력...,-0.899670,-3.996700e-01,neg,neg,1,하락,1,1
4,이데일리,20171208,http://www.edaily.co.kr/news/newspath.asp?news...,"디아이씨, 현기차 부품 공급업체 추가 선정이데일리 정병묵 기자 는 현대기아자동차 자...",2,5.483278e-01,0.451672,부품 공급업체 추가 선정 기자 공급업체 추가 선정 되 공시 하 [SEP] 습식 속 ...,0.096656,5.966557e-01,pos,pos,2,상승,3,1
5,연합뉴스,20171205,http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.as...,"현대기아쌍용한국지엠 8개 신차, 실내 공기질 합격점세종연합뉴스 이봉준 기자 최근 ...",3,1.059417e-02,0.989406,국지 엠 신차 실내 공기 합격 점 기자 제작 판매 되 차종 모두 실내 공기 관리 기...,-0.978812,2.118835e-02,neg,pos,3,중간,2,3
6,연합인포맥스,20171213,http://news.einfomax.co.kr/news/articleView.ht...,"현대차, 2025년 친환경차 38종으로 확대서울연합인포맥스 이민재 기자 현대자동차...",3,8.788295e-01,0.121170,종 확대 기자 오 차종 확대 하 전기차 매년 차종 추가 같 종 늘 계획 [SEP] ...,0.757659,1.757659e+00,pos,pos,0,매우상승,4,3
7,이데일리,20171204,http://www.edaily.co.kr/news/newspath.asp?news...,"이데일리 10 넘는 고금리 비중, 2년 7개월 만에 최고이데일리 1. 대형여행사들,...",2,9.830081e-01,0.016992,넘 고금리 비중 최고 [SEP] 단체 출시 임박 [SEP] 규제 부동산업 생산 감소...,0.966016,1.466016e+00,pos,pos,0,매우상승,4,1
8,연합뉴스,20171205,http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.as...,아이 채소 안 먹는 습관 고치려면강요 말고 반복적 노출서울연합뉴스 황희경 기자 누...,3,2.147025e-07,1.000000,채소 안 먹 습관 고치 강요 반복 노출 기자 덩어리 짜 자극 음식 신선 채소 좋 안...,-1.000000,4.294051e-07,neg,pos,3,중간,2,3
9,이데일리,20171206,http://www.edaily.co.kr/news/newspath.asp?news...,"기아차, 디자인 확 바뀐 레이 렌더링 공개이데일리 김보경 기자 레이가 6년만에 확 ...",2,1.664537e-02,0.983355,디자인 확 바뀌 레이 렌더링 공개 기자 레이 확 달라지 [SEP] 출시 디자인 변경...,-0.966709,-4.667093e-01,neg,neg,1,하락,1,3


In [ ]:
df_gr = df2.groupby('date')[['kmm_label_n_n5', 'targetpm']].mean()
# 날짜별로 train_test_split
train_date, test_date = train_test_split(df_gr, test_size=0.1, random_state=2022)
train_date_list = train_date.index
test_date_list = test_date.index
train_data = df2[df2['date'].isin(train_date_list)][['mpck_token', 'kmm_label_n_n5', 'target', 'date','targetpm']]
test_data = df2[df2['date'].isin(test_date_list)][['mpck_token', 'kmm_label_n_n5', 'target' ,'date','targetpm']]
train_data.shape, test_data.shape

((9773, 5), (1113, 5))

In [ ]:
df2['target'].value_counts()

2    4629
1    3169
3    3025
0      32
4      31
Name: target, dtype: int64

In [ ]:
train_data_list = []
for i in range(len(train_data)):
  train_data_list.append([train_data.iloc[i,0], train_data.iloc[i,1]])
train_data_list

[['건넨 특활 비 추적 유의미한 용처 파악 방 현덕 기자 시절 특수 활동비 상납 하 혐의 받 전 국가 원장 재판 넘 [SEP] 건넨 특수 활동비 사용처 상당부분 규명 하 전하 [SEP] 사정 따르 중앙 지검 특수 부 검사 구속 만료 다가오 전임 원장 뇌물 공여 특정 범죄 중처 벌 법상 국고채 손실 횡령 혐의 기소 하 방침 [SEP] 초대 국정 원장 남 전 원장 매달 가량 마지막 국정 원장 전 원장 매달 상납 하 의혹 받 [SEP] 남 전 원장 압박 관제 시위 단체 이익 몰아준 혐의 전 원장 전 정무 특활 비 하 불법 여론조사 비용 대납 혐의 있 [SEP] 특활 비 접수창구 역할 하 문고리 인방 일원 비서관 체포 하 공개 수사 시작 하 전직 국정 원장 달 구속 하 [SEP] 함께 구속 영장 청구 되 전 원장 영장 기각 되 [SEP] 구속 상태 전임 원장 일단 기소 하 전 원장 연루자 조사 마친 뒤 함께 처리 하 방안 검토 중 [SEP] 흘러들 가 국정원 특활 비 썼 추적 중 유의미한 용처 확인 하 전하 [SEP] 부르 본인 확인 하 이전 구체적 밝힐 없 입장 [SEP] 특활 비 불법 여론조사 비용 사용 되 의혹 정무 달 일비 공개 소환 조사 하 사건 구속 수감 되 필요 하 때 부르 조사 하 알려졌 [SEP] 사건 연루 되 대해서 조만간 출석 통보 하 방침 [SEP] 검사 출신 제시 하 사실 관계 인정 알려졌 [SEP] 별도 원장 시절 국정원 특활 비 수수 의혹 받 자유 소환 되 [SEP] ',
  1],
 ['특징 주현 대차 인방 내림 [SEP] 판매 부진 파업 우려 함정 선 기자 인방 주가 글로벌 판매 부진 파업 우려 더 해지 약세 나타나 [SEP] 대비 [SEP] 내리 거래 되 [SEP] 는 대비 [SEP] 내리 는 [SEP] 내리 거래 중 [SEP] 글로벌 판매 부진 겪 [SEP] 국내외 공장 판매가격 모두 감소 하 [SEP] 차 판매가격 대비 [SEP] 감소 하 [SEP]  [SEP] 줄 집계 되 [SEP] 사드 해빙 분위기 판매 회복 기대 미치 못하 

In [ ]:
test_data_list = []
for i in range(len(test_data)):
  test_data_list.append([test_data.iloc[i,0], test_data.iloc[i,1]])
test_data_list

[['큐 공장 방문 세계 지위 기자 큐 공장 찾 세계 지위 강화 하 주문 하 밝히 [SEP] 장쑤 난 퉁시 있 큐 공장 사업 거점 [SEP] 공장 찾 인수 [SEP] 공장 생산 되 제품 공장 운영 현황 관하 설명 들 뒤 인수 초창기 어렵 사업 환경 각고 노력 업계 최고 수준 제품 원가경쟁력 갖추 공장 거듭난 치하 하 [SEP] 그러 기술 양산 하 제품 글로벌 최고 품질 경쟁력 확보 하 당부 하 [SEP] 또 아니 생산 하 제품 특장 점 잘 살 사업군 최고 경쟁력 확보 세계 사업 지위 강화 하 육성 하 강조 하 [SEP] 특히 장강 물결 물결 밀 명언 있 장강 위치 공장 미래 사업 이끄 크 물결 되 거듭 당부 하 [SEP] 아울러 현지 협력업체 고객 동반성 장 위하 관계 구축 사회 공헌 지속 노력 현지 사회적책임 다 하 적극적 실천 하 주문 하 [SEP] 사업 시작 하 공장 인수 하 [SEP] 공장 세계 발돋움 하 디딤돌 되 사업장 [SEP] 인수 생산 규모 [SEP] 확대 되 [SEP] 또 불량 인수 견줘 개선 되 가장 낮 수준 되 인 생산성 [SEP] 배 향상 되 큐 인수 되 활발 선진 기술 교류 통하 효과 품질향상 원가경쟁력 제고 이루 설명 하 [SEP] 사업 그룹 미래 먹거리 점찍 육성 사업 [SEP] 평소 남다르 사명감 사업 매진 [SEP] 인류 미래 위하 장기 고민 하 육성 하 사업 여겼 장차 하 또 크 크 비전 실천 강조 [SEP] 방중 경제 사절단 합류 하 예정 [SEP] 열 한중 비즈니스 포럼 참석 정재 만난다 [SEP] 또 첨단 생산 법인 방문 글로벌 공급 하 현황 살피 예정 [SEP] ',
  1],
 ['서구 공장 주변 악취 원인 규명 하 기자 서구 공장 주변 악취 민원 수년 그대로 다 외부 조사 의뢰 하 필요 있 주장 하 [SEP] 장 열 행정 사무 감사 악취 방지 민간 협의체 인근 주민 참여 확대 하 개최 문제점 청취 하 주장 하 [SEP] 보건 환경 일괄 의뢰 하 악취 시료 채취 오염 검사 외부 전문 의뢰 곳 데이터 비교 분석 하 강조

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 512 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 16
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
data_train = BERTDataset(train_data_list, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data_list, 0, 1, tok, max_len, True, False)

In [ ]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# checkpoint
from copy import deepcopy
best_acc = 0
best_acc_model = None 

# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {} acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1), train_acc / (batch_id+1)))
    print("epoch {} train acc {} acc {}".format(e+1, train_acc / (batch_id+1), train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

    if (test_acc / (batch_id+1)) > best_acc:
          best_acc = (test_acc / (batch_id+1))
          best_acc_model = deepcopy(model)  
    
    print("epoch {} test acc {} acc {}".format(e+1, test_acc / (batch_id+1), test_acc / (batch_id+1)))

  0%|          | 0/163 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.608107328414917 train acc 0.21875 acc 0.21875
epoch 1 train acc 0.285659509202454 acc 0.285659509202454


  0%|          | 0/19 [00:00<?, ?it/s]

epoch 1 test acc 0.38569078947368424 acc 0.38569078947368424


  0%|          | 0/163 [00:00<?, ?it/s]

...

  0%|          | 0/19 [00:00<?, ?it/s]

epoch 24 test acc 0.31743421052631576 acc 0.31743421052631576


  0%|          | 0/163 [00:00<?, ?it/s]

epoch 25 batch id 1 loss 0.27607595920562744 train acc 0.90625 acc 0.90625


KeyboardInterrupt: ignored

In [ ]:
best_acc

0.41694078947368424

In [ ]:
best_acc_model

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
bertmodel2= torch.load("/content/drive/MyDrive/뉴스원인분석_modeling/model/model_1226_KoBERT_ml128_ep50.pt")
bertmodel2.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
test_data

,mpck_token,kmm_label_n_n5,target,date,targetpm
14,큐 공장 방문 세계 지위 기자 큐 공장 찾 세계 지위 강화 하 주문 하 밝히 [SE...,1,2,20171212,3
15,서구 공장 주변 악취 원인 규명 하 기자 서구 공장 주변 악취 민원 수년 그대로 다...,3,2,20171212,3
193,매출 회복 기대 오희 기자 투자 증권 대해 실적 개선 매출 회복 기대 된다며 투자의...,4,3,20180116,3
194,넘 업계 협 [SEP] 남 기자 사상 넘 업계 노사 임금 협상 마무리 수순 접어들 ...,2,3,20180116,3
195,포토 공개 되 피 용익 기자 신차 을 최초 공개 하 [SEP] 는 현지 컨벤션 센터...,2,3,20180116,3
...,...,...,...,...,...
10439,파업 공화국 될라 코로나 억눌렸 노사 갈등 관리 하 기자 코로나 팬데믹 대유행 늘 ...,0,1,20220613,1
10444,화물연대 파업 선 기자 가 하락 하 종가 기준 내리 [SEP] 시장 포인트 따르 거...,0,1,20220613,1
10733,탈레반 치하 경제난 자연재해 인권 암흑 속 특파원 강경 수니파 무장 조직 탈레반 수...,1,2,20220802,3
10734,미 중 갈등 확대 우려 코스피 [SEP] 하락 후퇴 종합 기자 코스피 갈등 확대 우...,1,2,20220802,3


In [ ]:
# batch_size 수정 코드
batch_size = 16

test_data3 = test_data.copy()
test_data3 = test_data3.reset_index(drop=True )
test_data3['pred'] = ''

bertmodel2.eval()
unseen_values = test_data3[['mpck_token', 'kmm_label_n_n5']].values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = bertmodel2(token_ids, valid_length, segment_ids)
  for outn in range(len(out)) :
    pred = int(out[outn].argmax().cpu().numpy())
    idx = batch_size * batch_id + outn
    test_data3.iloc[idx, -1] = pred
    if idx % 5000 == 0 :
      print(idx, pred)
test_data3.head(20)

0it [00:00, ?it/s]

0 1


,mpck_token,kmm_label_n_n5,target,date,targetpm,pred
0,큐 공장 방문 세계 지위 기자 큐 공장 찾 세계 지위 강화 하 주문 하 밝히 [SE...,1,2,20171212,3,1
1,서구 공장 주변 악취 원인 규명 하 기자 서구 공장 주변 악취 민원 수년 그대로 다...,3,2,20171212,3,3
2,매출 회복 기대 오희 기자 투자 증권 대해 실적 개선 매출 회복 기대 된다며 투자의...,4,3,20180116,3,4
3,넘 업계 협 [SEP] 남 기자 사상 넘 업계 노사 임금 협상 마무리 수순 접어들 ...,2,3,20180116,3,1
4,포토 공개 되 피 용익 기자 신차 을 최초 공개 하 [SEP] 는 현지 컨벤션 센터...,2,3,20180116,3,3
5,첨단 공장 가동 시장 공략 가속 민관 기자 첨단 생산 법인 공장 상업 생산 위하 본...,2,3,20180116,3,3
6,형 세계 최초 공개 피 용익 기자 신형 드러나 [SEP] 는 현지 센터 열 국제 오...,3,3,20180116,3,3
7,첨단 공장 상업 가동 공략 가속 기자 첨단 생산 법인 공장 상업 생산 위하 가동 시...,2,3,20180116,3,2
8,신형 세계 최초 공개 재웅 기자 현지 센터 열 국제 오토 쇼 신형 현지 세계 최초 ...,2,3,20180116,3,2
9,실시간 치법 알려 스마트 칫솔 출시 [SEP] 산업부 최저임금 인상 양극화 해소 가...,4,3,20180116,3,4


In [24]:
# 뉴스별 라벨링
# 뉴스 빈도 + 날 빈도
test_data4 = test_data.copy()
test_data4['pred'] = test_data3['pred'].values 
test_data4['pred'] = test_data4['pred'].astype('int')

df_gr = test_data4.groupby('date')[['target', 'pred', 'targetpm']].mean()
df_date = df_gr.index
for idx in df_date:
  df_gr.loc[idx, 'cnt_pred'] = int(test_data4[test_data4['date'] == idx ]['pred'].value_counts().idxmax())
df_gr['pred_label'] = df_gr['cnt_pred'].apply(lambda x : int(np.round(x)))
df_gr['target'] = df_gr['target'].apply(lambda x : int(np.round(x)))
df_gr['pred_pm'] = df_gr['pred_label'].apply(lambda x: 1 if x< 2 else 3)



df_gr = df_gr.reset_index()
for idx in df_gr.index :
  if df_gr.loc[idx, 'target'] == df_gr.loc[idx, 'pred_label']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_5'] = temp_eval

for idx in df_gr.index :
  if df_gr.loc[idx, 'target'] == 0 :
    df_gr.loc[idx, 'target_3label'] = 1
  elif df_gr.loc[idx, 'target'] == 4 :
    df_gr.loc[idx, 'target_3label'] = 3
  else :
    df_gr.loc[idx, 'target_3label'] = int(df_gr.loc[idx, 'target'])

  if df_gr.loc[idx, 'pred_label'] == 0 :
    df_gr.loc[idx, 'pred_label_3label'] = 1
  elif df_gr.loc[idx, 'pred_label'] == 4 :
    df_gr.loc[idx, 'pred_label_3label'] = 3
  else :
    df_gr.loc[idx, 'pred_label_3label'] = int(df_gr.loc[idx, 'pred_label'])

df_gr['pred_label_3label'] = df_gr['pred_label_3label'].apply(lambda x : int(x))

for idx in df_gr.index :
  if df_gr.loc[idx, 'target_3label'] == df_gr.loc[idx, 'pred_label_3label']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_3'] = temp_eval

for idx in df_gr.index :
  if df_gr.loc[idx, 'targetpm'] == df_gr.loc[idx, 'pred_pm']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_pm'] = temp_eval



print('5중분류 정확도 :', round(df_gr['eval_5'].sum() / len(df_gr), 2), '\n')
print('3중분류 정확도 :', round(df_gr['eval_3'].sum() / len(df_gr), 2), '\n')
print('이진분류 정확도 :', round(df_gr['eval_pm'].sum() / len(df_gr), 2), '\n')

df_gr.head()

5중분류 정확도 : 0.24 

3중분류 정확도 : 0.32 

이진분류 정확도 : 0.52 



,date,target,pred,targetpm,cnt_pred,pred_label,pred_pm,eval_5,target_3label,pred_label_3label,eval_3,eval_pm
0,20171212,2,2.000000,3.0,1.0,1,1,False,2.0,1,False,False
1,20180116,3,2.636364,3.0,4.0,4,3,False,3.0,3,True,True
2,20180205,2,1.000000,1.0,1.0,1,1,False,2.0,1,False,True
3,20180213,1,1.764706,1.0,2.0,2,3,False,1.0,2,False,False
4,20180514,2,1.750000,1.0,1.0,1,1,False,2.0,1,False,True


In [ ]:
test_data3['pred'].value_counts()

1    493
4    259
3    197
2    164
Name: pred, dtype: int64

In [ ]:
df_gr['pred_label'].value_counts()

1    73
4    17
3    15
2     9
Name: pred_label, dtype: int64

In [ ]:
df_gr['target'].value_counts()

2    47
3    39
1    28
Name: target, dtype: int64

In [ ]:
df_gr['cnt_pred'].mean()

1.7894736842105263

In [ ]:
df_gr['eval_pm'].value_counts(normalize=True)

True     0.578947
False    0.421053
Name: eval_pm, dtype: float64

In [ ]:
df_gr[df_gr['eval_pm'] == True]['pred_label'].value_counts()

1    35
4    13
3    12
2     6
Name: pred_label, dtype: int64